In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time

In [20]:
def get_links(area, amount, USERNAME, PASSWORD):

    links = []
    pageStart = 0

    driver = webdriver.Chrome(executable_path = 'C:\Drivers\Chrome_driver\chromedriver')
    driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
    driver.maximize_window()

    user_name_box = driver.find_element_by_id('username')
    user_name_box.send_keys(USERNAME)

    password_box = driver.find_element_by_id('password')
    password_box.send_keys(PASSWORD)

    submit_box = driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "from__button--floating", " " ))]') 
    submit_box.click()

    while pageStart < amount:
        driver.get('https://www.linkedin.com/jobs/search/?f_JT=F&location=' + area + '&sortBy=DD&start=' + str(pageStart))
        time.sleep(3)
        data = driver.page_source
        start = [m.start() for m in re.finditer('"jobPosting":"urn:li:fs_normalized_jobPosting:', data)]
        end = [m.start() for m in re.finditer('jobPostingResolutionResult"', data)]
        for i in range(len(start)):
            links.append('https://www.linkedin.com/jobs/view/' + data[start[i] + 46:end[i] - 4])
        pageStart += 7

    driver.close()
    return links

In [22]:
def get_jobs(links):
    
    names = ['Job Title', 'Company', 'Location', 'Senority Level', 'Employment Type', 'Job Function', 'Industries']
    df = pd.DataFrame(columns = names)

    failures = []
    count = 0

    driver = webdriver.Chrome(executable_path = 'C:\Drivers\Chrome_driver\chromedriver')

    for link in links:
        driver.get(link)
        driver.maximize_window()
        try:
            element_present = EC.presence_of_element_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "topcard__title", " " ))]'))
            WebDriverWait(driver, 3).until(element_present)
            title = driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "topcard__title", " " ))]')
            df.at[count, 'Job Title'] = title.text
            company =  driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "topcard__flavor--black-link", " " ))]')
            df.at[count, 'Company'] = company.text
            location = driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "topcard__flavor--bullet", " " ))]')
            df.at[count, 'Location'] = location.text
            job = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "job-criteria__text--criteria", " " ))]')
            df.at[count, 'Senority Level'] = job[0].text
            df.at[count, 'Employment Type'] = job[1].text
            df.at[count, 'Job Function'] = job[2].text
            industries = job[3].text
            if len(job) > 4:
                idx = 4
                while idx < len(job):
                    industries += ', ' + job[idx].text
                    idx += 1
            df.at[count, 'Industries'] = industries
            count += 1
        except:
            failures.append(link)
    driver.close()
    df.to_csv(area + '.csv', index=False)
    return df

In [ ]:
username = input('Username: ')
password = input('Password: ')
links = get_links('DistrictOfColumbia', 500, username, password)
df = get_jobs(links)

In [ ]:
failures

In [18]:
df

,Job Title,Company,Location,Senority Level,Employment Type,Job Function,Industries
0,Senior Technical Lead (Azure Cloud Engineer),Yochana IT Solutions INC,"Salem, OR",Associate,Full-time,Other,"Information Technology and Services, Computer ..."
1,Account Manager,GTL,"Portland, OR",Mid-Senior level,Full-time,Sales,"Business Development, Telecommunications, Info..."
2,Account Manager,GTL,"Vancouver, WA",Mid-Senior level,Full-time,Sales,"Business Development, Telecommunications, Info..."
3,Account Manager,GTL,"Salem, OR",Mid-Senior level,Full-time,Sales,"Business Development, Telecommunications, Info..."
4,Life Enrichment Assistant,Miramont Pointe,"Clackamas, OR",Entry level,Full-time,Administrative,"Restaurants, Food & Beverages, Food Production"
...,...,...,...,...,...,...,...
418,Azure Cloud Engineer,Yochana IT Solutions INC,"Portland, OR",Entry level,Full-time,Other,"Information Technology and Services, Computer ..."
419,SQL Programmer analyst for State of Oregon,nLeague Services Inc.,"Salem, OR",Entry level,Full-time,Information Technology,"Information Technology and Services, Computer ..."
420,SAP BI Reporting Architect for S/4 HANA Transf...,SoftHQ Inc,"Hillsboro, OR",Entry level,Full-time,Information Technology,"Information Technology and Services, Computer ..."
421,Senior Accountant,LMC Construction,"Portland, OR",Associate,Full-time,Accounting/Auditing,"Finance, Construction"
